# Piece Extract from Felles
Using time codes and ssh, extract from the RAW equivital CSV files some sensors during each performance of clapping music.

In [1]:
import sys
import os
from fabric import Connection
import shutil

import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

In [2]:
%load_ext autoreload
%autoreload 1
%aimport qex

In [3]:
%reload_ext autoreload

In [4]:
def tap_cue_align(cue,sig_ex,sig_ID):
    # a function to take a segment of signal and the tapping cue to determing
    # the best shift value would allow alignment of signal to cue
    signal = sig_ex.copy()
        # make the signal excerpt corr compatible. Inclusing cutting the extreme peaks
    signal[signal.isna()] = 0
    M = signal.quantile(0.998)
    signal = signal/M
    signal[signal>1] = 1
    
    # cue must be sampled at the same steady rate as the signal exerpt
    sampleshift_s = cue.index.to_series().diff().median()
    length = np.min([len(signal),len(cue)])
    
    if signal.diff().abs().sum()<0.1: # signal is flat
        shifts.append(np.nan)
        print('sig_ex is too flat.')
        return
    else:
        fig = plt.figure(figsize=(15,6))
        ax1 = plt.subplot(311)
        signal.plot(label=sig_ID,ax=ax1,)
        cue.plot.line(y='cue',ax=ax1)
        ax1.set_title(sig_ID + ' synch alignment')
        ax1.legend()
        #plt.xlim(cue_range)
        
        ax2 = plt.subplot(312)
        CCC = ax2.xcorr(cue['cue'].iloc[:length], signal.iloc[:length], usevlines=True, maxlags=length-1, normed=True, lw=3)
        ax2.grid(True)
        ax2.set_xticklabels('')
        signal.index = signal.index + sampleshift_s*CCC[0][np.argmax(CCC[1])]
        
        ax1 = plt.subplot(313)
        signal.plot(label=sig_ID,ax=ax1,)
        cue.plot.line(y='cue',ax=ax1)
#         plt.xlim(cue_range)
        ax1.grid(True)
        ax1.set_title('shift '+ str(sampleshift_s*CCC[0][np.argmax(CCC[1])])+ ' s')
        #plt.saveas('')
        plt.show()

    shift_stats = {"s_corr0": CCC[1][CCC[0]==0][0], # alignment quality without adjustment,
                   "s_corr_offset": np.amax(CCC[1]),
                   "s_offset_samples": CCC[0][np.argmax(CCC[1])], # shifts
                   "s_offset_time": sampleshift_s*CCC[0][np.argmax(CCC[1])],
                   "Length_xcorr_samples": len(CCC[0]),
                   "Length_xcorr_time": len(CCC[0])*sampleshift_s,
                   "devID": sig_ID,
                   "auto_offset_time":sampleshift_s*CCC[0][np.argmax(CCC[1])],
                   "Full_CCC": CCC
    }
    return shift_stats

In [5]:
pwd

'/Users/finn/Desktop/Current_Projects/Stavanger/Analysis'

# Prepare the timing and location information for extraction

In [6]:
Cue_template=pd.read_csv('./Timing/Synch_Cue_template_100Hz.csv')

In [7]:
Piece_timings = pd.read_csv('./Timing/ctimes.csv',index_col = 'Event')
Piece_timings 

,RD,C1,C2,C3,C4,C5
Event,,,,,,
Pre-Audio,NaN,NaN,-421.378,NaN,NaN,-981.902
Warmup,-300.000,-348.482,-361.303,-52.946,-344.999,-386.462
Tuning,-58.745,-61.511,-49.122,-48.773,-48.983,-225.636
Synch Taps,0.000,0.000,0.000,0.000,0.000,0.000
Applause for Conductor,19.294,28.292,122.014,76.074,21.929,-30.137
Saevarud,43.907,44.900,3279.222,3288.046,30.147,15.336
Applause for Saevarud,280.462,279.156,3511.465,3518.249,265.054,246.166
MC pre Reich,285.523,294.284,268.682,218.914,274.913,257.826
Reich,673.822,723.922,648.230,579.166,653.881,694.972


In [8]:
pd.to_datetime('2023-02-15 12:05:36.897+0100') + pd.to_timedelta(-2310.393,unit='s')

Timestamp('2023-02-15 11:27:06.504000+0100', tz='pytz.FixedOffset(60)')

In [9]:
synch_times = {'RG':pd.to_datetime('2023-02-15 11:25:26.573+0100'),
               'RE':pd.to_datetime('2023-02-15 11:25:26.573+0100'),
                'RD':pd.to_datetime('2023-02-15 12:05:36.897+0100'), # 97.0 
                 'C1':pd.to_datetime('2023-02-16 9:33:18.324+0100'),
                 'C2':pd.to_datetime('2023-02-16 11:16:22.697+0100'),
                 'C3':pd.to_datetime('2023-02-17 9:30:51.841+0100'),
                 'C4':pd.to_datetime('2023-02-17 11:16:29.298+0100'),
                 'C5':pd.to_datetime('2023-02-18 16:10:29.583+0100')}

In [10]:
# translate Piece_timings to Piece_dt
Piece_dt = pd.DataFrame(index = Piece_timings.index, columns = Piece_timings.columns)
for col in Piece_timings.columns:
    Piece_dt[col] = pd.to_timedelta(Piece_timings[col],unit='s')+synch_times[col]
Piece_dt.loc[['Synch Taps',
       'Saevarud', 'Applause for Saevarud', 'Reich',
       'Applause for Reich','Arban',
       'Applause for Arban','Grieg', 'Applause for Grieg','Tsjajkovskij', 'Applause for Tsjajkovskij','Christophersen',
       'Applause for Christophersen', 'Vivaldi',
       'Applause for Vivaldi', 'Strauss', 'Applause for Strauss'],:]

,RD,C1,C2,C3,C4,C5
Event,,,,,,
Synch Taps,2023-02-15 12:05:36.897000+01:00,2023-02-16 09:33:18.324000+01:00,2023-02-16 11:16:22.697000+01:00,2023-02-17 09:30:51.841000+01:00,2023-02-17 11:16:29.298000+01:00,2023-02-18 16:10:29.583000+01:00
Saevarud,2023-02-15 12:06:20.804000+01:00,2023-02-16 09:34:03.224000+01:00,2023-02-16 12:11:01.919000+01:00,2023-02-17 10:25:39.887000+01:00,2023-02-17 11:16:59.445000+01:00,2023-02-18 16:10:44.919000+01:00
Applause for Saevarud,2023-02-15 12:10:17.359000+01:00,2023-02-16 09:37:57.480000+01:00,2023-02-16 12:14:54.162000+01:00,2023-02-17 10:29:30.090000+01:00,2023-02-17 11:20:54.352000+01:00,2023-02-18 16:14:35.749000+01:00
Reich,2023-02-15 12:16:50.719000+01:00,2023-02-16 09:45:22.246000+01:00,2023-02-16 11:27:10.927000+01:00,2023-02-17 09:40:31.007000+01:00,2023-02-17 11:27:23.179000+01:00,2023-02-18 16:22:04.555000+01:00
Applause for Reich,2023-02-15 12:18:44.479000+01:00,2023-02-16 09:47:08.398000+01:00,2023-02-16 11:28:59.707000+01:00,2023-02-17 09:42:22.649000+01:00,2023-02-17 11:29:12.464000+01:00,2023-02-18 16:23:52.550000+01:00
Arban,2023-02-15 12:19:57.492000+01:00,2023-02-16 09:49:40.438000+01:00,2023-02-16 11:31:46.883000+01:00,2023-02-17 09:46:39.855000+01:00,2023-02-17 11:33:54.611000+01:00,2023-02-18 16:27:22.069000+01:00
Applause for Arban,2023-02-15 12:28:39.679000+01:00,2023-02-16 09:58:23.988000+01:00,2023-02-16 11:40:36.651000+01:00,2023-02-17 09:55:04.367000+01:00,2023-02-17 11:42:07.445000+01:00,2023-02-18 16:35:50.569000+01:00
Grieg,2023-02-15 12:29:28.287000+01:00,2023-02-16 10:00:00.398000+01:00,2023-02-16 11:41:29.819000+01:00,2023-02-17 09:56:02.783000+01:00,2023-02-17 11:43:02.267000+01:00,2023-02-18 16:36:51.241000+01:00
Applause for Grieg,2023-02-15 12:34:48.919000+01:00,2023-02-16 10:05:22.898000+01:00,2023-02-16 11:46:51.683000+01:00,2023-02-17 10:01:29.165000+01:00,2023-02-17 11:48:21.147000+01:00,2023-02-18 16:42:14.191000+01:00


In [11]:
Piece_dt.to_csv('./Timing/EventDateTimes.csv')

In [48]:
Show_intervals = {'RG':[pd.to_datetime('2023-02-15 10:00:00.000000+0100'),pd.to_datetime('2023-02-15 11:15:00.000000+0100')],
                 'RE':[pd.to_datetime('2023-02-15 11:10:00.000000+0100'),pd.to_datetime('2023-02-15 11:27:06.504000+0100')],
                 'RD':[pd.to_datetime('2023-02-15 12:04:38.152000+0100'),pd.to_datetime('2023-02-15 13:02:12.591000+0100')],
                 'C1':[pd.to_datetime('2023-02-16 09:32:16.813000+0100'),pd.to_datetime('2023-02-16 10:35:54.760000+0100')],
                 'C2':[pd.to_datetime('2023-02-16 11:15:33.575000+0100'),pd.to_datetime('2023-02-16 12:15:18.567000+0100')],
                 'C3':[pd.to_datetime('2023-02-17 09:30:03.068000+0100'),pd.to_datetime('2023-02-17 10:29:56.399000+0100')],
                 'C4':[pd.to_datetime('2023-02-17 11:15:40.315000+0100'),pd.to_datetime('2023-02-17 12:15:02.293000+0100')],
                 'C5':[pd.to_datetime('2023-02-18 16:06:43.947000+0100'),pd.to_datetime('2023-02-18 17:10:32.401000+0100 ')]}

In [ ]:
player_order = ['VN102','VN107','VN106','VN108','VN101','VN105','VN103','VN104','VN205','VN203','VN202','VN206','VN209','VN210','VN207','VN208','VN201','VN204','VA305','VA307','VA308','VA301','VA306','VA302','VA303','VA304','LS406','LS407','LS408','LS405','LS401','LS403','LS402','LS410','LS404','WW505','WW502','WW503','WW506','WW501','WW504','WW507','BR604','BR603','BR602','BR606','BR607','BR605','BR601','PC704','PC705','PC703','PC701','PC702','PC707']